In [1]:
import os
import dspy
import json

from typing import List, Dict

In [8]:
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')

In [9]:
lm = dspy.LM(
        "openai/gpt-5-mini",
        model_type="responses",
        temperature=1.0,
        max_tokens=16000,
    )

dspy.settings.configure(lm=lm)

#lm = dspy.LM('openai/gpt-4o-mini', api_key=openai_api_key)
#dspy.configure(lm=lm)

In [5]:
with open("prompts/20250923_100122.md", "r", encoding="utf-8") as f:
    prompt_poesia_md = f.read()

In [10]:
resposta = lm(messages=[
    {"role": "system", "content": prompt_poesia_md},
    {"role": "user", "content": "Água"}]
)

In [ ]:
resposta_formatada = "\n\n".join(resposta)
print(resposta_formatada)

{
  "Formas e estados": [
    "gota",
    "poça",
    "lama",
    "neblina",
    "vapor",
    "gelo",
    "névoa",
    "orvalho",
    "geada",
    "açúcar de neve",
    "maré",
    "corrente",
    "lagoa",
    "riacho",
    "catarata",
    "espuma",
    "lago",
    "oceano",
    "ressaca",
    "baixo-mar",
    "borda líquida",
    "anel de gelo"
  ],
  "Ações e movimentos": [
    "escorrer",
    "pingar",
    "borbulhar",
    "sussurrar (a água sussurra)",
    "arrebentar",
    "deslizar",
    "infiltrar",
    "transbordar",
    "moldar",
    "erosionar",
    "espalhar",
    "mergulhar",
    "submergir",
    "flutuar",
    "evaporar",
    "condensar",
    "gotejar",
    "laminar",
    "circular",
    "engolfar"
  ],
  "Sensações e sons": [
    "frescor",
    "frio na pele",
    "umidade que pesa",
    "brilho molhado",
    "eco metálico",
    "chiado de chuva",
    "estrondo de queda",
    "riso líquido",
    "silêncio que molha",
    "sopro úmido",
    "batida rítmica",
    "sussurro 

In [11]:
lm.history

[{'prompt': None,
  'messages': [{'role': 'system',
    'content': '## FUNÇÃO\nVocê é um assistente de escrita criativa com foco em poesia.\n\n## TAREFA\nSua tarefa é gerar associações de palavras que gerem novas ideias para a pessoa que está escrevendo.\n\nAo receber uma palavra, responda com uma série de palavras e expressões relacionadas a ela. Organize essas palavras em categorias ou temas. Inclua quantas outras achar necessário, da forma que achar melhor.\n\n## FORMATO\nRetorne apenas o objeto JSON, sem comentários, markdown etc.'},
   {'role': 'user', 'content': 'Água'}],
  'kwargs': {},
  'response': ResponsesAPIResponse(id='resp_bGl0ZWxsbTpjdXN0b21fbGxtX3Byb3ZpZGVyOm9wZW5haTttb2RlbF9pZDpOb25lO3Jlc3BvbnNlX2lkOnJlc3BfNjhkMmIzZjQ2Njk4ODE5NGE0NzkxMTU5ZmM1MjNjMjgwZGJmODFkMGM0ZmY0MTI1', created_at=1758639092, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-mini-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_68d2b3f5cce08194bdcd0